In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf
import talib
import seaborn as sns
import pickle

%matplotlib inline
plt.style.use('dark_background')

In [18]:
def get_unique_symbs(df):
    unique_symbs = set()
    for i in list(df.columns):
        unique_symbs.add(i[0])
    return unique_symbs

In [8]:
"""Read tickers from official site"""
nasdaq_listed = pd.read_csv('data/nasdaqlisted.txt', delimiter="|")
other_listed = pd.read_csv('data/otherlisted.txt', delimiter="|")
all_symbols = set(other_listed['Symbol']).union(set(nasdaq_listed['Symbol']))
all_symbols_cleaned = set()

for tick in all_symbols:
    if not "$" in tick and not "." in tick:
        all_symbols_cleaned.add(tick)

print(len(nasdaq_listed), len(other_listed), len(all_symbols), len(all_symbols_cleaned))

3558 5296 8854 8244


In [13]:
"""Getting tickers with no NaN and high volume"""
tickers_names_aux = " ".join(all_symbols_cleaned)
hist_downloaded_aux = yf.download(tickers_names_aux, period="1mo", interval="1d", start=None, end=None, actions=False, rounding=True, group_by='ticker', threads=True)

tickers_volM= set()
hist_downloaded_aux = hist_downloaded_aux.dropna(axis=1, how='all')
for tick in get_unique_symbs(hist_downloaded_aux):
    if hist_downloaded_aux[tick,'Volume'].mean() > 1000000: #High volume
        tickers_volM.add(tick)
print(len(tickers_volM))

with open('data/all_symbols_vol.pickle', 'wb') as handle:
    pickle.dump(tickers_volM, handle)


2148


In [14]:
"""Get my data"""
with open('data/all_symbols_vol.pickle', 'rb') as handle:
    all_symbols_vol = pickle.load(handle)

tickers_names = " ".join(all_symbols_vol)
hist_downloaded = yf.download(tickers_names, period="3mo", interval="1d", start=None, end=None, actions=False, rounding=True, group_by='ticker', threads=True)
hist_downloaded.to_pickle('data/all3mo.pickle')

[*********************100%***********************]  2148 of 2148 completed

8 Failed downloads:
- WPM: No data found for this date range, symbol may be delisted
- BORR: No data found, symbol may be delisted
- STX: No data found for this date range, symbol may be delisted
- FNDF: No data found for this date range, symbol may be delisted
- SBAC: No data found for this date range, symbol may be delisted
- VIAC: No data found, symbol may be delisted
- SBSW: No data found, symbol may be delisted
- HIO: No data found for this date range, symbol may be delisted


In [20]:
%%time
"""Read my data"""
hist = pd.read_pickle('data/all3mo.pickle')
print(len(hist.columns))
hist = hist.dropna(axis=1, how='all')
print(len(hist.columns))

12888
12840
CPU times: user 4.41 s, sys: 78.1 ms, total: 4.48 s
Wall time: 4.5 s


In [25]:
hist = pd.read_pickle('data/all3mo.pickle')
for tick in get_unique_symbs(hist):
    describe = hist[tick,'Close'].describe()
    discount = (describe['75%']/describe['min'])
    discount_norm = (describe['75%']-describe['min'])/describe['75%']
    if discount > 8:
        print(discount, discount_norm, tick)

11.03305785123967 0.9093632958801499 ICD
9.861904761904762 0.8985997102848865 AGS
26.472222222222225 0.9622245540398742 BRZU
8.9671875 0.8884823139919847 CEQP
12.601214574898785 0.9206425702811245 CDEV
9.442815249266863 0.8940993788819875 GNPX
11.934701492537311 0.9162107237767704 SSL
8.198888888888888 0.8780322536929123 GPMT
18.448943661971832 0.9457963546139898 REML
8.5 0.8823529411764707 TOPS
13.145494830132941 0.9239283105792461 NAIL
12.929104477611942 0.9226551226551227 TYG
22.487198795180724 0.9555302548303922 JNUG
9.74079754601227 0.8973390017320106 MITT
10.839285714285714 0.9077429983525535 QEP
10.135674381484439 0.9013385826771654 GFL
10.183962264150944 0.9018063918480778 KOS
10.101449275362318 0.9010043041606886 TELL
10.3625 0.9034981905910736 CEN
11.48820754716981 0.9129542188462327 XAN
55.21449704142013 0.9818888144675151 GUSH
8.183531746031747 0.8778033701054674 TRTX
10.564424635332253 0.9053426916733786 DFEN
9.019931662870158 0.8891344150514553 LK
11.914583333333333 0.916

In [29]:
hist['CVE','Volume'].mean()

8852732.258064516